In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

df  = pd.read_csv('BAresults_May.csv')

In [16]:
#1.Remove duplicates

print('before removing duplicates: ', len(df))
df = df.drop_duplicates()
print('after removing duplicates: ', len(df))

before removing duplicates:  225
after removing duplicates:  197


In [17]:
# 2.Only include business analyst title  

print('before removing items in JobTitle: ', len(df))
df = df[df['JobTitle'].str.contains('Business|Analyst|Sr.|Jr.|-')]
print('after removing items in JobTitle: ', len(df))

before removing items in JobTitle:  197
after removing items in JobTitle:  196


In [18]:
#3.Delete parttime job, we only focus on full-time job

print('before removing items in JobTitle or Job summary: ', len(df))
df = df[(~ df['JobTitle'].str.contains('part-time|part time'))]
df = df[(~ df['Summary'].str.contains('part-time|part time'))]
print('after removing items in JobTitle or Job summary: ', len(df))

before removing items in JobTitle or Job summary:  196
after removing items in JobTitle or Job summary:  196


In [19]:
# 4. Get levels from job title, job summary. 
#    Entry level: 0-3 years, Mid level: 3-5 years, Senior level: above 5 years
def get_level(x):
    title = x['JobTitle']
    if 'Senior' in title or 'Sr' in title:
        return 'Senior'
    elif 'Junior' in title or 'Jr' in title:
        return 'Junior'
    elif 'Entry Level' in title:
        return 'Entry Level'
    else:
        nums = re.findall('(\d)\+?\syears', x['Summary'])
        if not nums:
            return 'Entry Level'
        num = np.sum([int(n) for n in nums])
        if num <= 3:
            return 'Entry Level'
        elif num > 3 and num <= 5:
            return 'Mid Level'
        return 'Senior'
        
df['Level'] = df.apply(get_level, axis=1)

In [21]:
# 5.Find out remote availability for each job

def get_availability(x):
    if 'Remote' in x['Location'] or 'remote' in x['Location']:
        return 'Yes'
    else:
        return 'No'
    
df['Remote Availability'] = df.apply(get_availability, axis=1)

In [22]:
# 6. Get state name from jobs. If there's no state name, it is a remote job. 

def get_location(x):
    if 'Remote' in x['Location'] or 'Remote' in x['JobTitle']:
        return 'Remote'
    try:
        return re.findall('[A-Z]{2}', x['Location'])[0]
    except:
        if 'Indiana' in x['Location']:
            return 'IN'
        elif 'Ohio' in x['Location']:
            return 'OH'
        elif 'Illinois' in x['Location']:
            return 'IL'
        elif 'Kentucky' in x['Location']:
            return 'KY'
        elif 'North Carolina' in x['Location']:
            return 'NC'
        elif 'Nevada' in x['Location']:
            return 'NV'      
        return 'Remote'

df['location'] = df.apply(get_location, axis=1)
#df[df['State']==None]
df.head(5)


,JobTitle,Company,Location,PostDate,ExtractDate,Salary,Summary,Level,Remote Availability,location
0,Junior Business Analyst/Junior Quality Assurance,Neural Strategic Solutions Inc,"Centreville, VA 20121",Posted16 days ago,5/30/2022,$28 - $32 an hour,Use data to understand business patterns and t...,Junior,No,VA
1,Business Analyst – Remote,InteliChart,"Remote in Charlotte, NC 28202",Posted4 days ago,5/30/2022,"$85,000 - $95,000 a year",Medical insurance is 100% company-paid for all...,Entry Level,Yes,Remote
2,Jr. Business Analyst,NET2ASPIRE,Remote,EmployerActive 3 days ago,5/30/2022,"$65,000 - $80,000 a year",Define configuration specifications and busine...,Junior,Yes,Remote
3,Business Analyst,SnappyHires,"Austin, TX+1 location",EmployerActive 3 days ago,5/30/2022,$50 - $70 an hour,These positions will perform advanced GAP/Busi...,Entry Level,No,TX
4,Business/Data Analyst (Entry Level),Agama Solutions,Remote,Posted30+ days ago,5/30/2022,$30 an hour,Role: Business/Data Analyst (Entry Level). 1+ ...,Entry Level,Yes,Remote


In [23]:
# 7. Get post date: ExtractDate – PostDays = Post date
# For example: 5/31/2022 is extract date, the job has been posted for 30 days, the post date will be 5/1/2022
import re
def function(x):
    if x=='PostedToday' or x=='PostedJust posted' or x=='Hiring ongoing':
        return 0
    else:
        return int(re.sub("\D","",x))
df['Pose_date'] = df['PostDate'].apply(lambda x:function(x))
def func1(x, y):
    return pd.to_datetime(y, format='%m/%d/%Y') - pd.Timedelta(days=x)
df['Pose_date'] = df.apply(lambda x:func1(x.Pose_date, x.ExtractDate), axis=1)
import datetime
#df.drop(['PoseDate'], axis=1,inplace=True)
df['Pose_date'] = df['Pose_date'].apply(lambda x: datetime.datetime.strftime(x,"%m/%d/%Y"))
df.head(5)

,JobTitle,Company,Location,PostDate,ExtractDate,Salary,Summary,Level,Remote Availability,location,Pose_date
0,Junior Business Analyst/Junior Quality Assurance,Neural Strategic Solutions Inc,"Centreville, VA 20121",Posted16 days ago,5/30/2022,$28 - $32 an hour,Use data to understand business patterns and t...,Junior,No,VA,05/14/2022
1,Business Analyst – Remote,InteliChart,"Remote in Charlotte, NC 28202",Posted4 days ago,5/30/2022,"$85,000 - $95,000 a year",Medical insurance is 100% company-paid for all...,Entry Level,Yes,Remote,05/26/2022
2,Jr. Business Analyst,NET2ASPIRE,Remote,EmployerActive 3 days ago,5/30/2022,"$65,000 - $80,000 a year",Define configuration specifications and busine...,Junior,Yes,Remote,05/27/2022
3,Business Analyst,SnappyHires,"Austin, TX+1 location",EmployerActive 3 days ago,5/30/2022,$50 - $70 an hour,These positions will perform advanced GAP/Busi...,Entry Level,No,TX,05/27/2022
4,Business/Data Analyst (Entry Level),Agama Solutions,Remote,Posted30+ days ago,5/30/2022,$30 an hour,Role: Business/Data Analyst (Entry Level). 1+ ...,Entry Level,Yes,Remote,04/30/2022


In [24]:
# 8.Create a new column to analyze salary. Depends on hourly pay, annualy pay and monthly pay. 
def func2(x):
    if 'hour' in x:
        return 'Hourly Pay'
    elif ('annualy' in x) or ('year' in x):
        return 'Annualy Pay'
    elif 'month' in x:
        return 'Monthly Pay'
df['TypeOfSalary'] = df['Salary'].apply(func2)
df['TypeOfSalary'].value_counts(dropna=False)

Annualy Pay    75
Hourly Pay     69
NaN            48
Monthly Pay     4
Name: TypeOfSalary, dtype: int64

In [25]:
# b.Extract the range of salary from string, create three columns: maximum pay, minium pay, medium pay.

def func3(x, y):
    if y=='Hourly Pay':
        index = 40*52
    elif y=='Monthly Pay':
        index = 12
    else:
        index = 1
    x = x.replace(',','')
    if 'Not Ava' in x:
        return x, x, x
    elif '-' in x:
        return index*int(re.sub("\D","",x.split('-')[0])), index*int(re.sub("\D","",x.split('-')[1])), index*1/2*(int(re.sub("\D","",x.split('-')[0]))+int(re.sub("\D","",x.split('-')[1])))
    elif 'up to' in x:
        return 'Not Available', index*int(re.sub("\D","",x.split('-')[0])), 'Not Available'
    else:
        return 'Not Available', 'Not Available', index*int(re.sub("\D","",x))
#for i in df['Salary']:
#    print(i)
#    print(func4(i))
df[['Minimum_Pay','Maximum_Pay', 'Median_Pay']] = df.apply(lambda x:func3(x.Salary, x.TypeOfSalary),axis=1, result_type="expand")
df.head(5)

,JobTitle,Company,Location,PostDate,ExtractDate,Salary,Summary,Level,Remote Availability,location,Pose_date,TypeOfSalary,Minimum_Pay,Maximum_Pay,Median_Pay
0,Junior Business Analyst/Junior Quality Assurance,Neural Strategic Solutions Inc,"Centreville, VA 20121",Posted16 days ago,5/30/2022,$28 - $32 an hour,Use data to understand business patterns and t...,Junior,No,VA,05/14/2022,Hourly Pay,58240,66560,62400
1,Business Analyst – Remote,InteliChart,"Remote in Charlotte, NC 28202",Posted4 days ago,5/30/2022,"$85,000 - $95,000 a year",Medical insurance is 100% company-paid for all...,Entry Level,Yes,Remote,05/26/2022,Annualy Pay,85000,95000,90000
2,Jr. Business Analyst,NET2ASPIRE,Remote,EmployerActive 3 days ago,5/30/2022,"$65,000 - $80,000 a year",Define configuration specifications and busine...,Junior,Yes,Remote,05/27/2022,Annualy Pay,65000,80000,72500
3,Business Analyst,SnappyHires,"Austin, TX+1 location",EmployerActive 3 days ago,5/30/2022,$50 - $70 an hour,These positions will perform advanced GAP/Busi...,Entry Level,No,TX,05/27/2022,Hourly Pay,104000,145600,124800
4,Business/Data Analyst (Entry Level),Agama Solutions,Remote,Posted30+ days ago,5/30/2022,$30 an hour,Role: Business/Data Analyst (Entry Level). 1+ ...,Entry Level,Yes,Remote,04/30/2022,Hourly Pay,Not Available,Not Available,62400


In [11]:
# 9.Clean the string, delete non-string value in job summary. 

def func5(x):
    x_list = x.split(' ')# first split the words by blankspace
    for xx in range(len(x_list)):
        x_list[xx] = re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a])","",x_list[xx])
    return ' '.join(x_list)
df['Summary'] = df['Summary'].apply(func5)
df.head(5)

,JobTitle,Company,Location,PostDate,ExtractDate,Salary,Summary,Level,RemoteAvailability,State,Pose_date,TypeOfSalary,Minimum_Pay,Maximum_Pay,Median_Pay
0,Junior Business Analyst/Junior Quality Assurance,Neural Strategic Solutions Inc,"Centreville, VA 20121",Posted16 days ago,5/30/2022,$28 - $32 an hour,Use data to understand business patterns and t...,Junior,No,VA,05/14/2022,Hourly Pay,58240.0,66560.0,62400.0
1,Business Analyst – Remote,InteliChart,"Remote in Charlotte, NC 28202",Posted4 days ago,5/30/2022,"$85,000 - $95,000 a year",Medical insurance is 100 companypaid for all e...,Entry,No,NC,05/26/2022,Annualy Pay,85000.0,95000.0,90000.0
2,Jr. Business Analyst,NET2ASPIRE,Remote,EmployerActive 3 days ago,5/30/2022,"$65,000 - $80,000 a year",Define configuration specifications and busine...,Entry,No,Remote,05/27/2022,Annualy Pay,65000.0,80000.0,72500.0
3,Business Analyst,SnappyHires,"Austin, TX+1 location",EmployerActive 3 days ago,5/30/2022,$50 - $70 an hour,These positions will perform advanced GAPBusin...,Entry,No,TX,05/27/2022,Hourly Pay,104000.0,145600.0,124800.0
4,Business/Data Analyst (Entry Level),Agama Solutions,Remote,Posted30+ days ago,5/30/2022,$30 an hour,Role BusinessData Analyst Entry Level 1 Years ...,Entry,No,Remote,04/30/2022,Hourly Pay,Not Available,Not Available,62400


In [26]:
# export to csv file
df.to_csv('BA_May_Cleaned.csv', index=False)